In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.AbsRel import AbsRel
from OpenSCENARIO2CR.ConversionStatistics import EAnalyzer
from OpenSCENARIO2CR.EsminiWrapper.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.EsminiWrapper.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason

In [ ]:
scenario_path = ""
run_viewer = False
print_plots = False
plots_step = 5
plot_limit = 20  # If non-null the renderer follows the ego vehicle
following_obstacle_index = 0

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = None
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)

In [ ]:
# Setup Converter
converter = Osc2CrConverter(
    delta_t=0.1,
    max_time=120.0,
    grace_time=1.0,
    ignored_level=EStoryBoardElementType.ACT,
    log_to_console=True,
    log_to_file="/home/michael/log.txt",
    goal_state_time_step=AbsRel(interval=Interval(-5, 5), is_absolute=False),
    goal_state_position_length=50,
    goal_state_position_width=10,
    ego_filter=re.compile(r".*[Ee]go.*"),
    keep_ego_vehicle=True,
    do_trim_scenario=False,
    analyzers=[EAnalyzer.STL_MONITOR, EAnalyzer.DRIVABILITY_CHECKER]
)

result = converter.run_conversion(scenario_path)
if isinstance(result, EFailureReason):
    print(f"Scenario failed due to {result}")

In [ ]:
# plot every second time_stamp of the scenario
if print_plots and not isinstance(result, EFailureReason):
    scenario, planning_problem_set, statistics = result
    for i in range(0, max([obst.prediction.final_time_step for obst in scenario.dynamic_obstacles]) + 1, plots_step):
        plt.figure(figsize=(5, 5))

        if plot_limit is None:
            rnd = MPRenderer()
        else:
            rnd = MPRenderer(plot_limits=[-plot_limit, plot_limit, -plot_limit, plot_limit],
                             draw_params={"focus_obstacle_id": scenario.obstacles[following_obstacle_index].obstacle_id})
        # plot the scenario at different time step
        scenario.draw(rnd, draw_params={'time_begin': i})
        # plot the planning problem set
        planning_problem_set.draw(rnd)
        rnd.render()